In [3]:
# 03_cleaning_silver.ipynb - VERSION CORRIGÉE
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType, TimestampType

spark = SparkSession.builder.appName("Cleaning_Silver").getOrCreate()

# CORRECTION: Utiliser le système de fichiers local au lieu de HDFS
BASE_PATH = "file:///tmp"
RAW_NOAA = f"{BASE_PATH}/raw/noaa/"
SILVER_NOAA = f"{BASE_PATH}/silver/noaa/"

RAW_USGS = f"{BASE_PATH}/raw/usgs/events/"
SILVER_USGS = f"{BASE_PATH}/silver/usgs/"

# Créer les répertoires de sortie si nécessaire
import os
os.makedirs("/tmp/silver/noaa", exist_ok=True)
os.makedirs("/tmp/silver/usgs", exist_ok=True)

# -------- NOAA (batch) ----------
print("🔧 Traitement des données NOAA...")

try:
    # Essayer différents formats et chemins pour NOAA
    df_noaa = None
    possible_noaa_paths = [
        f"{RAW_NOAA}/*.csv",
        f"{RAW_NOAA}/*.parquet",
        f"{RAW_NOAA}/data/*.csv", 
        f"{RAW_NOAA}/data/*.parquet",
        "/tmp/raw/noaa/weather_data.parquet"  # Chemin des données de test
    ]
    
    for path in possible_noaa_paths:
        try:
            if "csv" in path:
                df_noaa = spark.read.option("header", True).csv(path)
            else:
                df_noaa = spark.read.parquet(path)
            
            if df_noaa.count() > 0:
                print(f"✅ Données NOAA trouvées: {path}")
                break
            else:
                df_noaa = None
        except:
            continue
    
    # Si pas de données trouvées, créer des données d'exemple
    if df_noaa is None:
        print("⚠️  Aucune donnée NOAA trouvée, création de données d'exemple...")
        from datetime import datetime, timedelta
        import random
        
        sample_data = []
        base_date = datetime(2024, 1, 1)
        
        for i in range(100):
            date = base_date + timedelta(days=i)
            sample_data.append((
                date.strftime('%Y-%m-%d'),
                f"STATION_{i%5}",
                40.7128 + random.uniform(-1, 1),
                -74.0060 + random.uniform(-1, 1),
                15 + random.uniform(-5, 5)
            ))
        
        df_noaa = spark.createDataFrame(
            sample_data, 
            ["DATE", "STATION", "LATITUDE", "LONGITUDE", "TEMP"]
        )
        
except Exception as e:
    print(f"❌ Erreur lecture NOAA: {e}")
    # Créer un DataFrame vide avec le schéma attendu
    df_noaa = spark.createDataFrame([], "date: timestamp, station_id: string, latitude: double, longitude: double, temp_c: double")

# Afficher les informations sur les données NOAA
print(f"📊 Données NOAA: {df_noaa.count()} lignes, {len(df_noaa.columns)} colonnes")
print("Colonnes disponibles:", df_noaa.columns)
df_noaa.printSchema()

# Normalisation : rename, types, unités
# Mapping standard : date, station_id, city, country, temp_c, wind_ms, precip_mm, lat, lon

# Helper conversion function (°F -> °C)
def f_to_c(col):
    return (F.col(col) - F.lit(32.0)) * F.lit(5.0/9.0)

# Exemple flexible : chercher colonnes candidates
cols = df_noaa.columns
print("🔍 Recherche des colonnes de température...")

# Détection automatique des colonnes de température
if "TEMP" in cols:
    df_noaa = df_noaa.withColumn("temp_c", F.col("TEMP").cast(DoubleType()))
    print("  ✅ Colonne TEMP trouvée et convertie")
elif "temp_c" in cols:
    df_noaa = df_noaa.withColumn("temp_c", F.col("temp_c").cast(DoubleType()))
    print("  ✅ Colonne temp_c trouvée")
elif "tmax" in cols:
    df_noaa = df_noaa.withColumn("temp_c", F.col("tmax").cast(DoubleType()))
    print("  ✅ Colonne tmax utilisée comme température")
elif "temperature" in cols:
    df_noaa = df_noaa.withColumn("temp_c", F.col("temperature").cast(DoubleType()))
    print("  ✅ Colonne temperature trouvée")
else:
    # Si aucune colonne de température trouvée, en créer une
    df_noaa = df_noaa.withColumn("temp_c", F.lit(15.0).cast(DoubleType()))
    print("  ⚠️  Aucune colonne température trouvée, valeur par défaut utilisée")

# Gestion des dates
if "DATE" in cols:
    df_noaa = df_noaa.withColumn("date", F.to_timestamp("DATE"))
    print("  ✅ Colonne DATE convertie en timestamp")
elif "date" in cols:
    df_noaa = df_noaa.withColumn("date", F.to_timestamp("date"))
    print("  ✅ Colonne date convertie en timestamp")
elif "DATE_TIME" in cols:
    df_noaa = df_noaa.withColumn("date", F.to_timestamp("DATE_TIME"))
    print("  ✅ Colonne DATE_TIME convertie en timestamp")
else:
    # Si aucune date, utiliser la date actuelle
    df_noaa = df_noaa.withColumn("date", F.current_timestamp())
    print("  ⚠️  Aucune colonne date trouvée, date actuelle utilisée")

# Cast columns safely (exemples)
for c in ["WIND", "PRCP", "LATITUDE", "LONGITUDE", "wind_ms", "precip_mm"]:
    if c in df_noaa.columns:
        df_noaa = df_noaa.withColumn(c, F.col(c).cast(DoubleType()))
        print(f"  ✅ Colonne {c} convertie en double")

# Standardize column names
column_mapping = {
    "LATITUDE": "latitude",
    "LAT": "latitude", 
    "LONGITUDE": "longitude",
    "LON": "longitude",
    "LNG": "longitude",
    "STATION": "station_id",
    "STATION_ID": "station_id"
}

for old_col, new_col in column_mapping.items():
    if old_col in df_noaa.columns:
        df_noaa = df_noaa.withColumnRenamed(old_col, new_col)
        print(f"  ✅ Colonne renommée: {old_col} -> {new_col}")

# If temp in F, convert
if "temp_f" in df_noaa.columns:
    df_noaa = df_noaa.withColumn("temp_c", f_to_c("temp_f"))
    print("  ✅ Conversion °F -> °C effectuée")

# Fill or drop missing temperature rows
initial_count = df_noaa.count()
df_noaa = df_noaa.filter(F.col("date").isNotNull())
final_count = df_noaa.count()
print(f"  📊 Filtrage des dates: {initial_count} -> {final_count} lignes")

# Create canonical columns if missing
required_columns = ["date", "station_id", "latitude", "longitude", "temp_c"]
for col in required_columns:
    if col not in df_noaa.columns:
        if col == "station_id":
            df_noaa = df_noaa.withColumn("station_id", F.monotonically_increasing_id())
        elif col in ["latitude", "longitude"]:
            df_noaa = df_noaa.withColumn(col, F.lit(0.0))
        print(f"  ✅ Colonne {col} ajoutée")

# CORRECTION: Ajouter la colonne year pour le partitionnement
df_noaa = df_noaa.withColumn("year", F.year("date"))

# Write SILVER
print("💾 Sauvegarde des données NOAA Silver...")
df_noaa_silver = df_noaa.select("date", "station_id", "latitude", "longitude", "temp_c", "year")

# CORRECTION: Utiliser le nom de colonne directement dans partitionBy()
df_noaa_silver.write.mode("overwrite").partitionBy("year").parquet(SILVER_NOAA)

print(f"✅ NOAA -> SILVER sauvegardé: {SILVER_NOAA}")
print(f"   {df_noaa_silver.count()} enregistrements traités")

# -------- USGS cleaning ----------
print("\n🔧 Traitement des données USGS...")

try:
    # Lecture des données USGS
    df_usgs = spark.read.parquet(RAW_USGS)
    print(f"📊 Données USGS: {df_usgs.count()} séismes trouvés")
    
except Exception as e:
    print(f"❌ Erreur lecture USGS: {e}")
    print("ℹ️  Création de données USGS d'exemple...")
    
    # Créer des données USGS d'exemple
    from datetime import datetime, timedelta
    import random
    
    sample_usgs = []
    base_time = datetime(2024, 1, 1)
    
    for i in range(50):
        event_time = base_time + timedelta(hours=i)
        sample_usgs.append((
            f"event_{i}",
            random.uniform(1.0, 5.0),
            event_time,
            random.uniform(-90, 90),
            random.uniform(-180, 180),
            f"Place {i}",
            random.uniform(0, 100)
        ))
    
    df_usgs = spark.createDataFrame(
        sample_usgs,
        ["event_id", "mag", "event_time", "latitude", "longitude", "place", "depth_km"]
    )

# Ensure columns exist and types are correct
df_usgs = df_usgs.withColumn("magnitude", F.col("mag").cast(DoubleType())) \
                 .withColumn("event_time", F.col("event_time").cast(TimestampType())) \
                 .withColumn("latitude", F.col("latitude").cast(DoubleType())) \
                 .withColumn("longitude", F.col("longitude").cast(DoubleType())) \
                 .withColumn("depth_km", F.col("depth_km").cast(DoubleType())) \
                 .withColumn("place", F.col("place"))

# Drop records without lat/lon
initial_usgs_count = df_usgs.count()
df_usgs = df_usgs.filter(F.col("latitude").isNotNull() & F.col("longitude").isNotNull())
final_usgs_count = df_usgs.count()

print(f"📊 Filtrage USGS: {initial_usgs_count} -> {final_usgs_count} séismes (lat/lon non-nulls)")

# Add human readable date
df_usgs = df_usgs.withColumn("date", F.to_date("event_time"))

# Write SILVER
print("💾 Sauvegarde des données USGS Silver...")
df_usgs_silver = df_usgs.select("event_id", "magnitude", "event_time", "date", "latitude", "longitude", "depth_km", "place")
df_usgs_silver.write.mode("overwrite").partitionBy("date").parquet(SILVER_USGS)

print(f"✅ USGS -> SILVER sauvegardé: {SILVER_USGS}")
print(f"   {df_usgs_silver.count()} séismes traités")

# Affichage des résultats finaux
print("\n🎉 RÉSUMÉ DU NETTOYAGE SILVER:")
print(f"📈 NOAA: {df_noaa_silver.count()} enregistrements météo")
print(f"🌋 USGS: {df_usgs_silver.count()} séismes")
print("\n📊 Aperçu des données NOAA:")
df_noaa_silver.show(5)
print("\n📊 Aperçu des données USGS:")
df_usgs_silver.show(5)

# Vérification des partitions créées
print("\n📁 VÉRIFICATION DES PARTITIONS:")
print("NOAA partitions (par année):")
df_noaa_silver.select("year").distinct().orderBy("year").show()

print("USGS partitions (par date):")
df_usgs_silver.select("date").distinct().orderBy("date").show()

🔧 Traitement des données NOAA...
⚠️  Aucune donnée NOAA trouvée, création de données d'exemple...
📊 Données NOAA: 100 lignes, 5 colonnes
Colonnes disponibles: ['DATE', 'STATION', 'LATITUDE', 'LONGITUDE', 'TEMP']
root
 |-- DATE: string (nullable = true)
 |-- STATION: string (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- TEMP: double (nullable = true)

🔍 Recherche des colonnes de température...
  ✅ Colonne TEMP trouvée et convertie
  ✅ Colonne DATE convertie en timestamp
  ✅ Colonne LATITUDE convertie en double
  ✅ Colonne LONGITUDE convertie en double
  ✅ Colonne renommée: LATITUDE -> latitude
  ✅ Colonne renommée: LONGITUDE -> longitude
  ✅ Colonne renommée: STATION -> station_id
  📊 Filtrage des dates: 100 -> 100 lignes
💾 Sauvegarde des données NOAA Silver...
✅ NOAA -> SILVER sauvegardé: file:///tmp/silver/noaa/
   100 enregistrements traités

🔧 Traitement des données USGS...
❌ Erreur lecture USGS: [UNABLE_TO_INFER_SCHEMA] Unabl